In [38]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import datetime as dt
import networkx as nx

import math
from decimal import Decimal, getcontext
getcontext().prec = 50

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'USDC-WETH 500 10'

# Functions

In [39]:
def Get_Price(_tick_):
    return Decimal(1/(1.0001**_tick_*10**-12))

# Read and Clean Data

Liquidity provision action

In [40]:
lp_action = pd.read_csv(os.path.join(data_dir, "uniswapv3_lp_actions", pool_name + "_LP.csv"))

# Final cleaning
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime(lp_action['BLOCK_TIMESTAMP'])
lp_action['AMOUNT0_ADJUSTED'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT0_ADJUSTED'], lp_action['AMOUNT0_ADJUSTED'])
lp_action['AMOUNT0_USD'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT0_USD'], lp_action['AMOUNT0_USD'])
lp_action['AMOUNT1_ADJUSTED'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT1_ADJUSTED'], lp_action['AMOUNT1_ADJUSTED'])
lp_action['AMOUNT1_USD'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT1_USD'], lp_action['AMOUNT1_USD'])
lp_action['L'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['L'], lp_action['L'])

# Round Liquidity Parameter to 5 decimal point
lp_action['L'] = round(lp_action['L'], 5)

lp_action.head()

,POOL_NAME,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,NF_TOKEN_ID,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER,L
0,USDC-WETH 500 10,Increase Liquidity,12376729,2021-05-05 21:42:11,0xb2ef52180d1e5f4835f4e343251286fa84743456,977.0,2995.507735,USDC,2995.292327,1.000000,WETH,3437.435736,191150,198080,2499.913599,4998.918171,345.07310
1,USDC-WETH 500 10,Decrease Liquidity,12376958,2021-05-05 22:27:41,0xb2ef52180d1e5f4835f4e343251286fa84743456,977.0,-2588.187276,USDC,-2588.001158,-0.931484,WETH,-3201.917248,191150,198080,2499.913599,4998.918171,-310.56579
2,USDC-WETH 500 10,Increase Liquidity,12377035,2021-05-05 22:44:11,0x35f5a1ec10deee1256351e37bc78dc80345895f8,1242.0,1.000000,USDC,0.999928,0.000146,WETH,0.500964,194990,195010,3398.182813,3404.985639,25.65271
3,USDC-WETH 500 10,Increase Liquidity,12377266,2021-05-05 23:34:39,0x9f9b987becfe15aca93ce9208cca146f252c8d46,1428.0,598.469729,USDC,598.426693,2.600000,WETH,8937.332900,186730,195460,3248.661521,7777.223709,447.99460
4,USDC-WETH 500 10,Increase Liquidity,12377308,2021-05-05 23:44:49,0x9f9b987becfe15aca93ce9208cca146f252c8d46,1464.0,3405.610348,USDC,3405.365449,0.000000,WETH,0.000000,194660,194990,3404.985639,3519.218937,3508.22475


Swaps action

In [41]:
swaps = pd.DataFrame(columns = ['BLOCK_NUMBER','BLOCK_TIMESTAMP','TX_HASH','SENDER','TX_TO','PLATFORM','POOL_NAME','CONTRACT_ADDRESS',
                                'EVENT_NAME','AMOUNT_IN','SYMBOL_IN','AMOUNT_OUT','SYMBOL_OUT'])

for i in range(1,4):
    temp = pd.read_parquet(os.path.join(data_dir, "dex_swaps", pool_name + "_" + str(i) + ".gzip"))
    swaps = pd.concat([swaps,temp], axis = 0)
    del temp

# Final cleaning
swaps.reset_index(drop = True, inplace = True)
swaps['BLOCK_NUMBER'] = swaps['BLOCK_NUMBER'].astype(int)
swaps['AMOUNT_IN'] = swaps['AMOUNT_IN'].astype(float)
swaps['AMOUNT_OUT'] = swaps['AMOUNT_OUT'].astype(float)
swaps['BLOCK_TIMESTAMP'] = pd.to_datetime(swaps['BLOCK_TIMESTAMP'])

# Since the pool charges a transaction fee of 5 basis point on the incoming token, and stored it elsewhere (not in liquidity pool), we subtract that
#swaps['AMOUNT_IN'] = swaps['AMOUNT_IN']*(1-0.0005)

# Changing signs of addition and extraction
swaps['AMOUNT_OUT'] = -swaps['AMOUNT_OUT']
swaps['AMOUNT0_ADJUSTED'] = np.where(swaps['SYMBOL_IN'] == 'WETH', swaps['AMOUNT_OUT'], swaps['AMOUNT_IN'])
swaps['TOKEN0_SYMBOL'] = 'USDC'
swaps['AMOUNT1_ADJUSTED'] = np.where(swaps['SYMBOL_IN'] == 'WETH', swaps['AMOUNT_IN'], swaps['AMOUNT_OUT'])
swaps['TOKEN1_SYMBOL'] = 'WETH'

swaps = swaps[['POOL_NAME','BLOCK_NUMBER','BLOCK_TIMESTAMP','SENDER','EVENT_NAME','AMOUNT0_ADJUSTED','TOKEN0_SYMBOL','AMOUNT1_ADJUSTED','TOKEN1_SYMBOL']]
swaps.sort_values(by = 'BLOCK_NUMBER', ascending = True, inplace = True)
swaps.reset_index(drop = True, inplace = True)

swaps.head()

,POOL_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,EVENT_NAME,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL
0,USDC-WETH 500 10,12376891,2021-05-05 22:15:01,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,-119.744094,USDC,0.035000,WETH
1,USDC-WETH 500 10,12377278,2021-05-05 23:37:55,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,499.756414,USDC,-0.144241,WETH
2,USDC-WETH 500 10,12377345,2021-05-05 23:51:36,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,365.000000,USDC,-0.103492,WETH
3,USDC-WETH 500 10,12377364,2021-05-05 23:56:34,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,-176.180828,USDC,0.050000,WETH
4,USDC-WETH 500 10,12377369,2021-05-05 23:57:36,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,-514.279985,USDC,0.146000,WETH


Merging

In [42]:
# Removing new LPs
lp_action = lp_action[lp_action['BLOCK_NUMBER'] <= swaps['BLOCK_NUMBER'].max()]
lp_action.reset_index(drop = True, inplace = True)

data = pd.concat([swaps, lp_action], axis = 0)
data.sort_values(by = 'BLOCK_NUMBER', ascending = True, inplace = True)
data.rename(columns = {'AMOUNT0_ADJUSTED':'Delta USDC', 'AMOUNT1_ADJUSTED':'Delta WETH'}, inplace = True)
data.reset_index(drop = True, inplace = True)
data = data[~((data['Delta WETH'] == 0) & (data['Delta USDC'] == 0))]

data = data[['POOL_NAME','BLOCK_NUMBER','BLOCK_TIMESTAMP','SENDER','EVENT_NAME','Delta WETH','Delta USDC','TICK_LOWER','TICK_UPPER','PRICE_LOWER','PRICE_UPPER','L']]

data.head()

,POOL_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,EVENT_NAME,Delta WETH,Delta USDC,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER,L
0,USDC-WETH 500 10,12376729,2021-05-05 21:42:11,0xb2ef52180d1e5f4835f4e343251286fa84743456,Increase Liquidity,1.000000,2995.507735,191150.0,198080.0,2499.913599,4998.918171,345.07310
1,USDC-WETH 500 10,12376891,2021-05-05 22:15:01,0xe592427a0aece92de3edee1f18e0157c05861564,Swap,0.035000,-119.744094,NaN,NaN,NaN,NaN,NaN
2,USDC-WETH 500 10,12376958,2021-05-05 22:27:41,0xb2ef52180d1e5f4835f4e343251286fa84743456,Decrease Liquidity,-0.931484,-2588.187276,191150.0,198080.0,2499.913599,4998.918171,-310.56579
3,USDC-WETH 500 10,12377035,2021-05-05 22:44:11,0x35f5a1ec10deee1256351e37bc78dc80345895f8,Increase Liquidity,0.000146,1.000000,194990.0,195010.0,3398.182813,3404.985639,25.65271
4,USDC-WETH 500 10,12377266,2021-05-05 23:34:39,0x9f9b987becfe15aca93ce9208cca146f252c8d46,Increase Liquidity,2.600000,598.469729,186730.0,195460.0,3248.661521,7777.223709,447.99460


# Descriptive Statistics

We will be building data descriptives here. One assumption we made here is that each address represents one single agent.

## Basic Descriptive

In [51]:
print("Data period:", data['BLOCK_TIMESTAMP'].min().strftime("%d %B %Y"), "-", data['BLOCK_TIMESTAMP'].max().strftime("%d %B %Y"))
print("Number of actions in our dataset:", len(data))
print("Number of liquidity provision action:", len(data[data['EVENT_NAME'].isin(['Increase Liquidity','Decrease Liquidity'])]))
print("Number of swap action:", len(data[data['EVENT_NAME'].isin(['Swap'])]))

Data period: 05 May 2021 - 12 May 2023
Number of actions in our dataset: 4929727
Number of liquidity provision action: 151554
Number of swap action: 4778173


# Liquidity Providers

## Wallet Issues

There is an issue where Liquidity Providers use one address for liquidity provision and the other for liquidity withdrawal.

We are able to identify these behaviours based on the ids of the NFT token mint, though these tokens are not address specific, if one id's NFT token has been minted and completely burned, i.e. liquidity parameter for that token is 0, we can safely assume that address that has surplus belongs to the same entity to address that has deficit.

In [46]:
cleared_tokens = lp_action[['NF_TOKEN_ID','L']].copy()
cleared_tokens = cleared_tokens.groupby(['NF_TOKEN_ID'])['L'].sum().reset_index()
cleared_tokens = cleared_tokens[np.abs(cleared_tokens['L']) < 1]
cleared_tokens = cleared_tokens['NF_TOKEN_ID']
cleared_tokens.head()

2    1428.0
3    1464.0
4    1485.0
5    1567.0
6    1714.0
Name: NF_TOKEN_ID, dtype: float64

We can demonstrate this by example.

In [47]:
address_clustering = lp_action[['SENDER','BLOCK_TIMESTAMP','NF_TOKEN_ID','L']].copy()
address_clustering = address_clustering.groupby(['SENDER','NF_TOKEN_ID'])['L'].sum().reset_index()
address_clustering.loc[np.abs(address_clustering['L']) < 1,'L'] = 0

# Get example
address_clustering = address_clustering[(address_clustering['L'] != 0) & (address_clustering['NF_TOKEN_ID'].isin(cleared_tokens))].sort_values(by = 'NF_TOKEN_ID')
address_clustering.head(10)

,SENDER,NF_TOKEN_ID,L
48178,0xbf68b3ac6f125ee6c98f1cba7d0f217274b21406,41090.0,-4.812185e+02
36503,0x895eee677eedfa433f63bfccea8b94a999e0bcbb,41090.0,4.812185e+02
45785,0xb3aad889bf41ec39571afa78aa5d9a26a05ddaab,42080.0,-3.508526e+04
34964,0x7f1f3e81ccf2fe1a8f6c111612b442a56c47f6d3,42080.0,3.508526e+04
25228,0x525208dd0b56c27bd10703bd675fca0509a17154,43099.0,7.367416e+03
473,0x0202ed9ff0d505f9b064a610199a001cef9977bd,43099.0,-7.367416e+03
6636,0x1335d0a7cba7e5742399b0c94305136e8f4bc2a8,45063.0,-3.251015e+06
4529,0x0ce93110c08c8e046e9d02e340c9ab744f185266,45063.0,3.251015e+06
42119,0xa52dba12f5f87215e209cb9ebfc792c52a328c0a,45653.0,-6.474513e+04
23879,0x4ad334cca43274ef8ea6b18a4557cb6657927589,45653.0,6.474513e+04


For the addresses above, we will change all address to a single address.

In [48]:
address_clustering = address_clustering.groupby(['NF_TOKEN_ID'])['SENDER'].apply(list).reset_index()
address_clustering = address_clustering['SENDER']

# Concat lists if they share common element
G = nx.Graph()
for l in address_clustering:
    G.add_edges_from(nx.utils.pairwise(l))
connected_components = list(nx.connected_components(G))
address_clustering = pd.DataFrame({'SENDER':[list(comp) for comp in connected_components]})

# Take one address as the shared address for all lists
address_clustering['ADDRESS'] = address_clustering['SENDER'].str[0]
address_clustering = address_clustering.explode('SENDER', ignore_index = True)
address_clustering = address_clustering[['ADDRESS','SENDER']].set_index('SENDER').to_dict()
address_clustering = address_clustering['ADDRESS']

lp_action['SENDER'] = lp_action['SENDER'].replace(address_clustering)

Final check on negative L.

In [49]:
negative_L_check = lp_action[['SENDER','BLOCK_TIMESTAMP','NF_TOKEN_ID','L']].copy()
negative_L_check = negative_L_check.groupby(['SENDER','NF_TOKEN_ID'])['L'].sum().reset_index()
negative_L_check.loc[np.abs(negative_L_check['L']) < 1,'L'] = 0
print(negative_L_check[negative_L_check['L'] < 0])

# Final bit of manual work
lp_action['SENDER'] = lp_action['SENDER'].replace({'0x6c340f813448f5b459e421fa1d385b8fceef6ca9':'0xdef4f7e2f82229776159bb2fb346e6ab5b0a6b08'})
lp_action.loc[lp_action['NF_TOKEN_ID'] == 187506.0,'SENDER'] = '0xd00fce4966821da1edd1221a02af0afc876365e4'

                                           SENDER  NF_TOKEN_ID           L
13691  0x33d1edf4e954661c84114fd6f2cd545e7283d7b9     187506.0   -15.29122
16386  0x3a43ace38aeaca3073901b14d97dadf660693e1f     187506.0   -89.48132
19001  0x455210daaff5378f74d894efb7719749d60e0fe3     187506.0  -573.05015
28990  0x610e0ae9e5802875130872b4376d9187cac67d62     187506.0  -135.47779
30766  0x6c340f813448f5b459e421fa1d385b8fceef6ca9     429783.0  -745.49640
47093  0xbeb730172e5f8cee9108b8dab3b1fa0069e45ecc     187506.0  -635.20442
47271  0xc03b7a303ff86ccdefe7a880568545d95925b230     187506.0  -220.40553
50251  0xd00fce4966821da1edd1221a02af0afc876365e4     187506.0 -7407.70188
51808  0xd5d6a475a38560661c9cfbf3137b6df7bcee83c4     187506.0  -760.30126


At last, we can check the number of unique liquidity providers in the Uniswap V3 WETH-USDC 500 10 pool.

In [50]:
print("Total Number of unique Liquidity Provider:", len(lp_action['SENDER'].unique()))

Total Number of unique Liquidity Provider: 9533


## Pool Concentration

Now, we are interested in the concentration of our liquidity pool.

In [ ]:
lp_pool_concentration = lp_action[['SENDER','BLOCK_TIMESTAMP','NF_TOKEN_ID','L']].copy()
lp_pool_concentration = lp_pool_concentration.groupby(['SENDER','NF_TOKEN_ID'])['L'].sum().reset_index()

# Solve rounding errors
lp_pool_concentration.loc[np.abs(lp_pool_concentration['L']) < 1,'L'] = 0
lp_pool_concentration[lp_pool_concentration['L'] < 0]

In [ ]:
lp_action[(lp_action['NF_TOKEN_ID'] == 429783.0)]

In [ ]:
lp_pool_concentration = lp_action[['SENDER','BLOCK_TIMESTAMP','L']].copy()
lp_pool_concentration = lp_pool_concentration.groupby(['SENDER','BLOCK_TIMESTAMP'])['L'].sum().reset_index()
lp_pool_concentration = lp_pool_concentration.pivot(index = 'BLOCK_TIMESTAMP', columns = 'SENDER', values = 'L')
lp_pool_concentration.cumsum(axis = 1).fillna()

# Traders

In [ ]:
print("Total Number of unique Traders:", len(swaps['SENDER'].unique()))